In [116]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import csv
import os

# training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [127]:

DATA_PATH = os.path.join('MP_Data')  # Path for exported data, numpy arrays


#getting dataset values from csv
with open('newlabelstrain.csv', newline='') as f:
    reader = csv.reader(f)
    data = [row[0] for row in reader]


actions = np.array(data)
no_sequences = 30  # thirty videos worth of data
sequence_length = 30  # each video is 30 frame in length

label_map = {label:num for num, label in enumerate(actions)}
print(label_map)


{'hi': 0, 'howAre': 1, 'you': 2}


In [128]:

sequences, labels = [], []  # x data, y data
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

x = np.array(sequences)
print(x.shape)
y = to_categorical(labels).astype(int)
print(y)


(90, 30, 1662)
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [129]:

# creating training partitions
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(85, 30, 1662) (5, 30, 1662) (85, 3) (5, 3)


In [130]:

# training

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential() # instantiating the model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [131]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])
print(model.summary())

Epoch 1/2000
3/3 [==============================] - 3s 234ms/step - loss: 1.2755 - categorical_accuracy: 0.3176
Epoch 2/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.2638 - categorical_accuracy: 0.2941
Epoch 3/2000
3/3 [==============================] - 0s 59ms/step - loss: 6.2075 - categorical_accuracy: 0.2706
Epoch 4/2000
3/3 [==============================] - 0s 58ms/step - loss: 20.4784 - categorical_accuracy: 0.2824
Epoch 5/2000
3/3 [==============================] - 0s 61ms/step - loss: 1.5728 - categorical_accuracy: 0.3059
Epoch 6/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.0749 - categorical_accuracy: 0.4353
Epoch 7/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.7465 - categorical_accuracy: 0.6706
Epoch 8/2000
3/3 [==============================] - 0s 70ms/step - loss: 0.7245 - categorical_accuracy: 0.6706
Epoch 9/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.6040 - categorical_accuracy: 0.7176

3/3 [==============================] - 0s 65ms/step - loss: 0.5016 - categorical_accuracy: 0.9765
Epoch 75/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.3377 - categorical_accuracy: 0.9882
Epoch 76/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.2097 - categorical_accuracy: 0.9882
Epoch 77/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.1857 - categorical_accuracy: 0.9647
Epoch 78/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2558 - categorical_accuracy: 0.9176
Epoch 79/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.9300 - categorical_accuracy: 0.7059
Epoch 80/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5210 - categorical_accuracy: 0.8353
Epoch 81/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.5134 - categorical_accuracy: 0.8000
Epoch 82/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.4071 - categorical_accuracy: 0.8824
Epoch 

3/3 [==============================] - 0s 65ms/step - loss: 4.8568e-05 - categorical_accuracy: 1.0000
Epoch 146/2000
3/3 [==============================] - 0s 62ms/step - loss: 4.7651e-05 - categorical_accuracy: 1.0000
Epoch 147/2000
3/3 [==============================] - 0s 62ms/step - loss: 4.6631e-05 - categorical_accuracy: 1.0000
Epoch 148/2000
3/3 [==============================] - 0s 61ms/step - loss: 4.5738e-05 - categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 61ms/step - loss: 4.4975e-05 - categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 62ms/step - loss: 4.4057e-05 - categorical_accuracy: 1.0000
Epoch 151/2000
3/3 [==============================] - 0s 60ms/step - loss: 4.3173e-05 - categorical_accuracy: 1.0000
Epoch 152/2000
3/3 [==============================] - 0s 63ms/step - loss: 4.2387e-05 - categorical_accuracy: 1.0000
Epoch 153/2000
3/3 [==============================] - 0s 64ms/step - loss: 4.15

3/3 [==============================] - 0s 97ms/step - loss: 1.8731e-05 - categorical_accuracy: 1.0000
Epoch 216/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.8531e-05 - categorical_accuracy: 1.0000
Epoch 217/2000
3/3 [==============================] - 0s 90ms/step - loss: 1.8341e-05 - categorical_accuracy: 1.0000
Epoch 218/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.8143e-05 - categorical_accuracy: 1.0000
Epoch 219/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.7963e-05 - categorical_accuracy: 1.0000
Epoch 220/2000
3/3 [==============================] - 0s 93ms/step - loss: 1.7770e-05 - categorical_accuracy: 1.0000
Epoch 221/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.7592e-05 - categorical_accuracy: 1.0000
Epoch 222/2000
3/3 [==============================] - 0s 91ms/step - loss: 1.7383e-05 - categorical_accuracy: 1.0000
Epoch 223/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.7

3/3 [==============================] - 0s 113ms/step - loss: 1.0431e-05 - categorical_accuracy: 1.0000
Epoch 285/2000
3/3 [==============================] - 0s 118ms/step - loss: 1.0357e-05 - categorical_accuracy: 1.0000
Epoch 286/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.0277e-05 - categorical_accuracy: 1.0000
Epoch 287/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0220e-05 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 0s 109ms/step - loss: 1.0148e-05 - categorical_accuracy: 1.0000
Epoch 289/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0086e-05 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0018e-05 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 0s 109ms/step - loss: 9.9587e-06 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 111ms/step - l

...

3/3 [==============================] - 0s 133ms/step - loss: 4.6772e-06 - categorical_accuracy: 1.0000
Epoch 423/2000
3/3 [==============================] - 0s 135ms/step - loss: 4.6561e-06 - categorical_accuracy: 1.0000
Epoch 424/2000
3/3 [==============================] - 0s 124ms/step - loss: 4.6351e-06 - categorical_accuracy: 1.0000
Epoch 425/2000
3/3 [==============================] - 0s 123ms/step - loss: 4.6127e-06 - categorical_accuracy: 1.0000
Epoch 426/2000
3/3 [==============================] - 0s 131ms/step - loss: 4.5972e-06 - categorical_accuracy: 1.0000
Epoch 427/2000
3/3 [==============================] - 0s 132ms/step - loss: 4.5748e-06 - categorical_accuracy: 1.0000
Epoch 428/2000
3/3 [==============================] - 0s 125ms/step - loss: 4.5509e-06 - categorical_accuracy: 1.0000
Epoch 429/2000
3/3 [==============================] - 0s 137ms/step - loss: 4.5327e-06 - categorical_accuracy: 1.0000
Epoch 430/2000
3/3 [==============================] - 0s 129ms/step - l

3/3 [==============================] - 0s 133ms/step - loss: 3.4360e-06 - categorical_accuracy: 1.0000
Epoch 492/2000
3/3 [==============================] - 0s 150ms/step - loss: 3.4220e-06 - categorical_accuracy: 1.0000
Epoch 493/2000
3/3 [==============================] - 0s 128ms/step - loss: 3.4108e-06 - categorical_accuracy: 1.0000
Epoch 494/2000
3/3 [==============================] - 0s 137ms/step - loss: 3.3995e-06 - categorical_accuracy: 1.0000
Epoch 495/2000
3/3 [==============================] - 0s 129ms/step - loss: 3.3869e-06 - categorical_accuracy: 1.0000
Epoch 496/2000
3/3 [==============================] - 0s 135ms/step - loss: 3.3673e-06 - categorical_accuracy: 1.0000
Epoch 497/2000
3/3 [==============================] - 0s 127ms/step - loss: 3.3589e-06 - categorical_accuracy: 1.0000
Epoch 498/2000
3/3 [==============================] - 0s 134ms/step - loss: 3.3420e-06 - categorical_accuracy: 1.0000
Epoch 499/2000
3/3 [==============================] - 0s 137ms/step - l

3/3 [==============================] - 0s 144ms/step - loss: 2.6142e-06 - categorical_accuracy: 1.0000
Epoch 561/2000
3/3 [==============================] - 0s 141ms/step - loss: 2.6058e-06 - categorical_accuracy: 1.0000
Epoch 562/2000
3/3 [==============================] - 0s 140ms/step - loss: 2.5931e-06 - categorical_accuracy: 1.0000
Epoch 563/2000
3/3 [==============================] - 0s 128ms/step - loss: 2.5819e-06 - categorical_accuracy: 1.0000
Epoch 564/2000
3/3 [==============================] - 0s 142ms/step - loss: 2.5693e-06 - categorical_accuracy: 1.0000
Epoch 565/2000
3/3 [==============================] - 0s 135ms/step - loss: 2.5637e-06 - categorical_accuracy: 1.0000
Epoch 566/2000
3/3 [==============================] - 0s 135ms/step - loss: 2.5525e-06 - categorical_accuracy: 1.0000
Epoch 567/2000
3/3 [==============================] - 0s 134ms/step - loss: 2.5426e-06 - categorical_accuracy: 1.0000
Epoch 568/2000
3/3 [==============================] - 0s 144ms/step - l

3/3 [==============================] - 0s 161ms/step - loss: 1.9915e-06 - categorical_accuracy: 1.0000
Epoch 630/2000
3/3 [==============================] - 0s 155ms/step - loss: 1.9859e-06 - categorical_accuracy: 1.0000
Epoch 631/2000
3/3 [==============================] - 0s 161ms/step - loss: 1.9775e-06 - categorical_accuracy: 1.0000
Epoch 632/2000
3/3 [==============================] - 0s 146ms/step - loss: 1.9733e-06 - categorical_accuracy: 1.0000
Epoch 633/2000


KeyboardInterrupt: 

In [132]:
res = model.predict(x_test)
print(res)
print(actions[np.argmax(res[0])])
for i in range(len(res)):
    print(actions[np.argmax(res[i])])

[[9.9999881e-01 9.4954481e-07 2.1322836e-07]
 [9.9999976e-01 2.2785315e-07 1.8757327e-19]
 [9.5576747e-09 5.5102523e-10 1.0000000e+00]
 [1.3404036e-07 1.1264548e-08 9.9999988e-01]
 [9.9999833e-01 1.5570929e-06 7.7443289e-08]]
hi
hi
hi
you
you
hi


In [133]:
# save weights
model.save('action.h5')

In [134]:
#evaluation using confusion matrix
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
matrix = multilabel_confusion_matrix(ytrue, yhat)
print(matrix)
accuracy_score = accuracy_score(ytrue, yhat)
print(f"{accuracy_score= }")


[[[2 0]
  [0 3]]

 [[3 0]
  [0 2]]]
accuracy_score= 1.0
